In [ ]:
%matplotlib inline 
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns

Note the location pattern on GCS : gs://downscaled-288ec5ac/diagnostics/RELEASE-v{delivery_version}/{diagnostics_name}/{activity_id}/{institution_id}/{source_id}/{experiment_id}/{member_id}/{table_id}/{variable_id}/{delivery_version}.zarr

ERA-5
gs://downscaled-288ec5ac/diagnostics/RELEASE-{delivery_version}/{diagnostics_name}/reanalysis/ERA5/F320/{variable_id}/{delivery_version}.zarr

In [ ]:
'''read_gcs_zarr('gs://downscaled-48ec31ab/diagnostics/RELEASE-v1.1/clean-daily-precip-diagnostics/ScenarioMIP/BCC/BCC-CSM2-MR/ssp370/r1i1p1f1/day/pr/v1.1.zarr')'''

In [ ]:
plot_dir = '/gcs/impactlab-data/climate/downscaling/paper/'

In [ ]:
import itertools
import colorcet as cc
clrs = itertools.cycle(sns.color_palette(cc.glasbey, n_colors=17))

In [ ]:
sns.color_palette(cc.glasbey, n_colors=17)

In [ ]:
model = 'NorESM2-LM'
institution = 'NCC'

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 4))

# colors = cm.rainbow(np.linspace(0, 1, len(tasmax_hist_clean.city.values)))
# for i, variable in enumerate(['tasmax', 'tasmin', 'precip']):
for i, variable in enumerate(['tasmax']):
    # for i, variable in enumerate(['tasmax', 'tasmin']):
    
    if variable == "precip":
        file_var_name = "pr"
    else:
        file_var_name = variable
    
    var_ssp370_ds = xr.open_zarr('gs://downscaled-48ec31ab/diagnostics/RELEASE-v1.1/daily-{}-diagnostics/ScenarioMIP/BCC/BCC-CSM2-MR/ssp370/r1i1p1f1/day/{}/v1.1.zarr'.format(variable, file_var_name))
    var_hist_ds = xr.open_zarr('gs://downscaled-48ec31ab/diagnostics/RELEASE-v1.1/daily-{}-diagnostics/CMIP/BCC/BCC-CSM2-MR/historical/r1i1p1f1/day/{}/v1.1.zarr'.format(variable, file_var_name))

    var_ssp370_clean = xr.open_zarr('gs://downscaled-48ec31ab/diagnostics/RELEASE-v1.1/clean-daily-{}-diagnostics/ScenarioMIP/BCC/BCC-CSM2-MR/ssp370/r1i1p1f1/day/{}/v1.1.zarr'.format(variable, file_var_name))
    var_hist_clean = xr.open_zarr('gs://downscaled-48ec31ab/diagnostics/RELEASE-v1.1/clean-daily-{}-diagnostics/CMIP/BCC/BCC-CSM2-MR/historical/r1i1p1f1/day/{}/v1.1.zarr'.format(variable, file_var_name))

    var_ref = xr.open_zarr('gs://downscaled-48ec31ab/diagnostics/RELEASE-v1.1/daily-{}-diagnostics/reanalysis/ERA5/F320/{}/v1.1.zarr'.format(variable, file_var_name))

    city_trend_error = {}
    city_absolute_error = {}
    for city in var_hist_clean.city.values:
        
        if variable == 'precip':
            var = 'pr'
        else:
            var = variable 

        sim_trend = (var_ssp370_clean.sel(city=city)[var].sel(time=slice('2080', '2099')).values - var_hist_clean.sel(city=city)          [var].sel(time=slice('1995', '2014')).values)

        downscale_trend = (var_ssp370_ds.sel(city=city)[var].sel(time=slice('2080', '2099')).values - var_hist_ds.sel(city=city)          [var].sel(time=slice('1995', '2014')).values)

        city_error = np.median(np.abs(downscale_trend - sim_trend))
        city_trend_error[city] = city_error

        sim_hist = (var_hist_ds.sel(city=city)[var].sel(time=slice('1995', '2014')).values - var_ref.sel(city=city)[var].sel(time=slice('1995', '2014')).values)

        abs_error = np.median(np.abs(sim_hist))

        city_absolute_error[city] = abs_error
        
    
        axes[i].plot(city_absolute_error[city], city_trend_error[city], 'o', c=next(clrs), label=city)
    axes[i].set_xlabel('Bias adjustment')
    axes[0].set_ylabel('Trend preservation')
    
    axes[2].legend(loc='center left', bbox_to_anchor=(1, 0.5))
    
    axes[i].set_title(variable)
    
plt.savefig(os.path.join(plot_dir, 'cities_abs_rel_seasonal_error.png'), 
            bbox_inches='tight', dpi=200)

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 4))

# colors = cm.rainbow(np.linspace(0, 1, len(tasmax_hist_clean.city.values)))
for i, variable in enumerate(['tasmax', 'tasmin', 'precip']):
    # for i, variable in enumerate(['precip']):
    # for i, variable in enumerate(['tasmax', 'tasmin']):
    
    if variable == "precip":
        file_var_name = "pr"
    else:
        file_var_name = variable
    
    var_ssp370_ds = xr.open_zarr('gs://downscaled-48ec31ab/diagnostics/RELEASE-v1.1/daily-{}-diagnostics/ScenarioMIP/BCC/BCC-CSM2-MR/ssp370/r1i1p1f1/day/{}/v1.1.zarr'.format(variable, file_var_name))
    var_hist_ds = xr.open_zarr('gs://downscaled-48ec31ab/diagnostics/RELEASE-v1.1/daily-{}-diagnostics/CMIP/BCC/BCC-CSM2-MR/historical/r1i1p1f1/day/{}/v1.1.zarr'.format(variable, file_var_name))

    var_ssp370_clean = xr.open_zarr('gs://downscaled-48ec31ab/diagnostics/RELEASE-v1.1/clean-daily-{}-diagnostics/ScenarioMIP/BCC/BCC-CSM2-MR/ssp370/r1i1p1f1/day/{}/v1.1.zarr'.format(variable, file_var_name))
    var_hist_clean = xr.open_zarr('gs://downscaled-48ec31ab/diagnostics/RELEASE-v1.1/clean-daily-{}-diagnostics/CMIP/BCC/BCC-CSM2-MR/historical/r1i1p1f1/day/{}/v1.1.zarr'.format(variable, file_var_name))

    var_ref = xr.open_zarr('gs://downscaled-48ec31ab/diagnostics/RELEASE-v1.1/daily-{}-diagnostics/reanalysis/ERA5/F320/{}/v1.1.zarr'.format(variable, file_var_name))

    city_trend_error = {}
    city_absolute_error = {}
    for city in var_hist_clean.city.values:
        
        if city == 'Cairo' or city == 'Miami' or city == 'Istanbul' or city == 'New York' or city == 'Mexico City':
            pass
        else: 
            
            if variable == 'precip':
                var = 'pr'
            else:
                var = variable 

            if variable == 'precip':
                sim_trend = var_ssp370_clean.sel(city=city)[var].sel(time=slice('2080', '2099')).resample(time='QS-DEC').sum().values / var_hist_clean.sel(city=city)          [var].sel(time=slice('1995', '2014')).resample(time='QS-DEC').sum().values

                downscale_trend = var_ssp370_ds.sel(city=city)[var].sel(time=slice('2080', '2099')).resample(time='QS-DEC').sum().values / var_hist_ds.sel(city=city)          [var].sel(time=slice('1995', '2014')).resample(time='QS-DEC').sum().values

                sim_hist = var_hist_ds.sel(city=city)[var].sel(time=slice('1995', '2014')).resample(time='QS-DEC').sum().values / var_ref.sel(city=city)[var].sel(time=slice('1995', '2014')).resample(time='QS-DEC').sum().values

                city_error = np.median(np.abs(downscale_trend / sim_trend))
            else: 

                sim_trend = var_ssp370_clean.sel(city=city)[var].sel(time=slice('2080', '2099')).resample(time='QS-DEC').mean().values - var_hist_clean.sel(city=city)          [var].sel(time=slice('1995', '2014')).resample(time='QS-DEC').mean().values

                downscale_trend = var_ssp370_ds.sel(city=city)[var].sel(time=slice('2080', '2099')).resample(time='QS-DEC').mean().values - var_hist_ds.sel(city=city)          [var].sel(time=slice('1995', '2014')).resample(time='QS-DEC').mean().values

                sim_hist = var_hist_ds.sel(city=city)[var].sel(time=slice('1995', '2014')).resample(time='QS-DEC').mean().values - var_ref.sel(city=city)[var].sel(time=slice('1995', '2014')).resample(time='QS-DEC').mean().values

                city_error = np.median(np.abs(downscale_trend - sim_trend))


            city_trend_error[city] = city_error
            abs_error = np.median(np.abs(sim_hist))

            city_absolute_error[city] = abs_error


            axes[i].plot(city_absolute_error[city], city_trend_error[city], 'o', c=next(clrs), label=city)
        axes[i].set_xlabel('Bias adjustment')
        axes[0].set_ylabel('Trend preservation')
        axes[i].set_ylim([0, 2.5])

        axes[2].legend(loc='center left', bbox_to_anchor=(1, 0.5))
        if variable == 'precip': 
            title_variable = 'precipitation'
        elif variable == 'tasmax':
            title_variable = 'maximum temperature'
        else:
            title_variable = 'minimum temperature'
        axes[i].set_title(title_variable)

plt.savefig(os.path.join(plot_dir, 'cities_abs_rel_seasonal_error.png'), 
            bbox_inches='tight', dpi=200)

In [ ]:
fig, axes = plt.subplots(1, 5, figsize=(25, 4))

for i, q in enumerate([0.05, 0.25, 0.5, 0.75, 0.95]):
    variable = 'tasmax'
    
    
    if variable == "precip":
        file_var_name = "pr"
    else:
        file_var_name = variable
    
    var_ssp370_ds = xr.open_zarr('gs://downscaled-48ec31ab/diagnostics/RELEASE-v1.1/daily-{}-diagnostics/ScenarioMIP/BCC/BCC-CSM2-MR/ssp370/r1i1p1f1/day/{}/v1.1.zarr'.format(variable, file_var_name))
    var_hist_ds = xr.open_zarr('gs://downscaled-48ec31ab/diagnostics/RELEASE-v1.1/daily-{}-diagnostics/CMIP/BCC/BCC-CSM2-MR/historical/r1i1p1f1/day/{}/v1.1.zarr'.format(variable, file_var_name))

    var_ssp370_clean = xr.open_zarr('gs://downscaled-48ec31ab/diagnostics/RELEASE-v1.1/clean-daily-{}-diagnostics/ScenarioMIP/BCC/BCC-CSM2-MR/ssp370/r1i1p1f1/day/{}/v1.1.zarr'.format(variable, file_var_name))
    var_hist_clean = xr.open_zarr('gs://downscaled-48ec31ab/diagnostics/RELEASE-v1.1/clean-daily-{}-diagnostics/CMIP/BCC/BCC-CSM2-MR/historical/r1i1p1f1/day/{}/v1.1.zarr'.format(variable, file_var_name))

    var_ref = xr.open_zarr('gs://downscaled-48ec31ab/diagnostics/RELEASE-v1.1/daily-{}-diagnostics/reanalysis/ERA5/F320/{}/v1.1.zarr'.format(variable, file_var_name))

    city_trend_error = {}
    city_absolute_error = {}
    for city in var_hist_clean.city.values:
        
        if variable == 'precip':
            var = 'pr'
        else:
            var = variable 

        sim_trend = (var_ssp370_clean.sel(city=city)[var].sel(time=slice('2080', '2099')).groupby('time.year').quantile(q).values 
                     - var_hist_clean.sel(city=city)[var].sel(time=slice('1995', '2014')).groupby('time.year').quantile(q).values)

        downscale_trend = (var_ssp370_ds.sel(city=city)[var].sel(time=slice('2080', '2099')).groupby('time.year').quantile(q).values 
                           - var_hist_ds.sel(city=city)[var].sel(time=slice('1995', '2014')).groupby('time.year').quantile(q).values)

        city_error = np.median(np.abs(downscale_trend - sim_trend))
        city_trend_error[city] = city_error

        sim_hist = (var_hist_ds.sel(city=city)[var].sel(time=slice('1995', '2014')).groupby('time.year').quantile(q).values 
                    - var_ref.sel(city=city)[var].sel(time=slice('1995', '2014')).groupby('time.year').quantile(q).values)

        abs_error = np.median(np.abs(sim_hist))

        city_absolute_error[city] = abs_error
        
    
        axes[i].plot(city_absolute_error[city], city_trend_error[city], 'o', c=next(clrs), label=city)
    axes[i].set_xlabel('Bias adjustment')
    axes[i].set_xlim([0, 16])
    axes[i].set_ylim([0, 2.5])
    axes[0].set_ylabel('Trend preservation')
    
    axes[4].legend(loc='center left', bbox_to_anchor=(1, 0.5))
    
    axes[i].set_title('{}'.format(q))

In [ ]:
fig, axes = plt.subplots(1, 5, figsize=(25, 4))

for i, q in enumerate([0.05, 0.25, 0.5, 0.75, 0.95]):
    variable = 'precip'
    
    
    if variable == "precip":
        file_var_name = "pr"
    else:
        file_var_name = variable
    
    var_ssp370_ds = xr.open_zarr('gs://downscaled-48ec31ab/diagnostics/RELEASE-v1.1/daily-{}-diagnostics/ScenarioMIP/BCC/BCC-CSM2-MR/ssp370/r1i1p1f1/day/{}/v1.1.zarr'.format(variable, file_var_name))
    var_hist_ds = xr.open_zarr('gs://downscaled-48ec31ab/diagnostics/RELEASE-v1.1/daily-{}-diagnostics/CMIP/BCC/BCC-CSM2-MR/historical/r1i1p1f1/day/{}/v1.1.zarr'.format(variable, file_var_name))

    var_ssp370_clean = xr.open_zarr('gs://downscaled-48ec31ab/diagnostics/RELEASE-v1.1/clean-daily-{}-diagnostics/ScenarioMIP/BCC/BCC-CSM2-MR/ssp370/r1i1p1f1/day/{}/v1.1.zarr'.format(variable, file_var_name))
    var_hist_clean = xr.open_zarr('gs://downscaled-48ec31ab/diagnostics/RELEASE-v1.1/clean-daily-{}-diagnostics/CMIP/BCC/BCC-CSM2-MR/historical/r1i1p1f1/day/{}/v1.1.zarr'.format(variable, file_var_name))

    var_ref = xr.open_zarr('gs://downscaled-48ec31ab/diagnostics/RELEASE-v1.1/daily-{}-diagnostics/reanalysis/ERA5/F320/{}/v1.1.zarr'.format(variable, file_var_name))

    city_trend_error = {}
    city_absolute_error = {}
    for city in var_hist_clean.city.values:
        
        if variable == 'precip':
            var = 'pr'
        else:
            var = variable 

        sim_trend = (var_ssp370_clean.sel(city=city)[var].sel(time=slice('2080', '2099')).groupby('time.year').quantile(q).values 
                     - var_hist_clean.sel(city=city)[var].sel(time=slice('1995', '2014')).groupby('time.year').quantile(q).values)

        downscale_trend = (var_ssp370_ds.sel(city=city)[var].sel(time=slice('2080', '2099')).groupby('time.year').quantile(q).values 
                           - var_hist_ds.sel(city=city)[var].sel(time=slice('1995', '2014')).groupby('time.year').quantile(q).values)

        city_error = np.median(np.abs(downscale_trend - sim_trend))
        city_trend_error[city] = city_error

        sim_hist = (var_hist_ds.sel(city=city)[var].sel(time=slice('1995', '2014')).groupby('time.year').quantile(q).values 
                    - var_ref.sel(city=city)[var].sel(time=slice('1995', '2014')).groupby('time.year').quantile(q).values)

        abs_error = np.median(np.abs(sim_hist))

        city_absolute_error[city] = abs_error
        
    
        axes[i].plot(city_absolute_error[city], city_trend_error[city], 'o', c=next(clrs), label=city)
    axes[i].set_xlabel('Bias adjustment')
    axes[i].set_xlim([0, 16])
    axes[i].set_ylim([0, 16])
    axes[0].set_ylabel('Trend preservation')
    
    axes[4].legend(loc='center left', bbox_to_anchor=(1, 0.5))
    
    axes[i].set_title('{}'.format(q))

In [ ]:
fig, axes = plt.subplots(1, 5, figsize=(25, 4))

for i, q in enumerate([0.05, 0.25, 0.5, 0.75, 0.95]):
    variable = 'tasmax'
    
    
    if variable == "precip":
        file_var_name = "pr"
    else:
        file_var_name = variable
    
    var_ssp370_ds = xr.open_zarr('gs://downscaled-48ec31ab/diagnostics/RELEASE-v1.1/daily-{}-diagnostics/ScenarioMIP/BCC/BCC-CSM2-MR/ssp370/r1i1p1f1/day/{}/v1.1.zarr'.format(variable, file_var_name))
    var_hist_ds = xr.open_zarr('gs://downscaled-48ec31ab/diagnostics/RELEASE-v1.1/daily-{}-diagnostics/CMIP/BCC/BCC-CSM2-MR/historical/r1i1p1f1/day/{}/v1.1.zarr'.format(variable, file_var_name))

    var_ssp370_clean = xr.open_zarr('gs://downscaled-48ec31ab/diagnostics/RELEASE-v1.1/clean-daily-{}-diagnostics/ScenarioMIP/BCC/BCC-CSM2-MR/ssp370/r1i1p1f1/day/{}/v1.1.zarr'.format(variable, file_var_name))
    var_hist_clean = xr.open_zarr('gs://downscaled-48ec31ab/diagnostics/RELEASE-v1.1/clean-daily-{}-diagnostics/CMIP/BCC/BCC-CSM2-MR/historical/r1i1p1f1/day/{}/v1.1.zarr'.format(variable, file_var_name))

    var_ref = xr.open_zarr('gs://downscaled-48ec31ab/diagnostics/RELEASE-v1.1/daily-{}-diagnostics/reanalysis/ERA5/F320/{}/v1.1.zarr'.format(variable, file_var_name))

    city_trend_error = {}
    city_absolute_error = {}
    for city in var_hist_clean.city.values:
        
        if variable == 'precip':
            var = 'pr'
        else:
            var = variable 

        sim_trend = (var_ssp370_clean.sel(city=city)[var].sel(time=slice('2080', '2099')).chunk(dict(time=-1)).groupby('time.season').quantile(q).values 
                     - var_hist_clean.sel(city=city)[var].sel(time=slice('1995', '2014')).chunk(dict(time=-1)).groupby('time.season').quantile(q).values)

        downscale_trend = (var_ssp370_ds.sel(city=city)[var].sel(time=slice('2080', '2099')).chunk(dict(time=-1)).groupby('time.season').quantile(q).values 
                           - var_hist_ds.sel(city=city)[var].sel(time=slice('1995', '2014')).chunk(dict(time=-1)).groupby('time.season').quantile(q).values)

        city_error = np.median(np.abs(downscale_trend - sim_trend))
        city_trend_error[city] = city_error

        sim_hist = (var_hist_ds.sel(city=city)[var].sel(time=slice('1995', '2014')).chunk(dict(time=-1)).groupby('time.season').quantile(q).values 
                    - var_ref.sel(city=city)[var].sel(time=slice('1995', '2014')).chunk(dict(time=-1)).groupby('time.season').quantile(q).values)

        abs_error = np.median(np.abs(sim_hist))

        city_absolute_error[city] = abs_error
        
    
        axes[i].plot(city_absolute_error[city], city_trend_error[city], 'o', c=next(clrs), label=city)
    axes[i].set_xlabel('Bias adjustment')
    axes[i].set_xlim([0, 16])
    axes[i].set_ylim([0, 2.5])
    axes[0].set_ylabel('Trend preservation')
    
    axes[4].legend(loc='center left', bbox_to_anchor=(1, 0.5))
    
    axes[i].set_title('{}'.format(q))

In [ ]:
fig, axes = plt.subplots(1, 5, figsize=(25, 4))

for i, q in enumerate([0.05, 0.25, 0.5, 0.75, 0.95]):
    variable = 'tasmin'
    
    
    if variable == "precip":
        file_var_name = "pr"
    else:
        file_var_name = variable
    
    var_ssp370_ds = xr.open_zarr('gs://downscaled-48ec31ab/diagnostics/RELEASE-v1.1/daily-{}-diagnostics/ScenarioMIP/BCC/BCC-CSM2-MR/ssp370/r1i1p1f1/day/{}/v1.1.zarr'.format(variable, file_var_name))
    var_hist_ds = xr.open_zarr('gs://downscaled-48ec31ab/diagnostics/RELEASE-v1.1/daily-{}-diagnostics/CMIP/BCC/BCC-CSM2-MR/historical/r1i1p1f1/day/{}/v1.1.zarr'.format(variable, file_var_name))

    var_ssp370_clean = xr.open_zarr('gs://downscaled-48ec31ab/diagnostics/RELEASE-v1.1/clean-daily-{}-diagnostics/ScenarioMIP/BCC/BCC-CSM2-MR/ssp370/r1i1p1f1/day/{}/v1.1.zarr'.format(variable, file_var_name))
    var_hist_clean = xr.open_zarr('gs://downscaled-48ec31ab/diagnostics/RELEASE-v1.1/clean-daily-{}-diagnostics/CMIP/BCC/BCC-CSM2-MR/historical/r1i1p1f1/day/{}/v1.1.zarr'.format(variable, file_var_name))

    var_ref = xr.open_zarr('gs://downscaled-48ec31ab/diagnostics/RELEASE-v1.1/daily-{}-diagnostics/reanalysis/ERA5/F320/{}/v1.1.zarr'.format(variable, file_var_name))

    city_trend_error = {}
    city_absolute_error = {}
    for city in var_hist_clean.city.values:
        
        if variable == 'precip':
            var = 'pr'
        else:
            var = variable 

        sim_trend = (var_ssp370_clean.sel(city=city)[var].sel(time=slice('2080', '2099')).chunk(dict(time=-1)).groupby('time.season').quantile(q).values 
                     - var_hist_clean.sel(city=city)[var].sel(time=slice('1995', '2014')).chunk(dict(time=-1)).groupby('time.season').quantile(q).values)

        downscale_trend = (var_ssp370_ds.sel(city=city)[var].sel(time=slice('2080', '2099')).chunk(dict(time=-1)).groupby('time.season').quantile(q).values 
                           - var_hist_ds.sel(city=city)[var].sel(time=slice('1995', '2014')).chunk(dict(time=-1)).groupby('time.season').quantile(q).values)

        city_error = np.median(np.abs(downscale_trend - sim_trend))
        city_trend_error[city] = city_error

        sim_hist = (var_hist_ds.sel(city=city)[var].sel(time=slice('1995', '2014')).chunk(dict(time=-1)).groupby('time.season').quantile(q).values 
                    - var_ref.sel(city=city)[var].sel(time=slice('1995', '2014')).chunk(dict(time=-1)).groupby('time.season').quantile(q).values)

        abs_error = np.median(np.abs(sim_hist))

        city_absolute_error[city] = abs_error
        
    
        axes[i].plot(city_absolute_error[city], city_trend_error[city], 'o', c=next(clrs), label=city)
    axes[i].set_xlabel('Bias adjustment')
    axes[i].set_xlim([0, 2.5])
    axes[i].set_ylim([0, 2.5])
    axes[0].set_ylabel('Trend preservation')
    
    axes[4].legend(loc='center left', bbox_to_anchor=(1, 0.5))
    
    axes[i].set_title('{}'.format(q))

In [ ]:
fig, axes = plt.subplots(1, 5, figsize=(25, 4))

for i, q in enumerate([0.05, 0.25, 0.5, 0.75, 0.95]):
    variable = 'precip'
    
    
    if variable == "precip":
        file_var_name = "pr"
    else:
        file_var_name = variable
    
    var_ssp370_ds = xr.open_zarr('gs://downscaled-48ec31ab/diagnostics/RELEASE-v1.1/daily-{}-diagnostics/ScenarioMIP/BCC/BCC-CSM2-MR/ssp370/r1i1p1f1/day/{}/v1.1.zarr'.format(variable, file_var_name))
    var_hist_ds = xr.open_zarr('gs://downscaled-48ec31ab/diagnostics/RELEASE-v1.1/daily-{}-diagnostics/CMIP/BCC/BCC-CSM2-MR/historical/r1i1p1f1/day/{}/v1.1.zarr'.format(variable, file_var_name))

    var_ssp370_clean = xr.open_zarr('gs://downscaled-48ec31ab/diagnostics/RELEASE-v1.1/clean-daily-{}-diagnostics/ScenarioMIP/BCC/BCC-CSM2-MR/ssp370/r1i1p1f1/day/{}/v1.1.zarr'.format(variable, file_var_name))
    var_hist_clean = xr.open_zarr('gs://downscaled-48ec31ab/diagnostics/RELEASE-v1.1/clean-daily-{}-diagnostics/CMIP/BCC/BCC-CSM2-MR/historical/r1i1p1f1/day/{}/v1.1.zarr'.format(variable, file_var_name))

    var_ref = xr.open_zarr('gs://downscaled-48ec31ab/diagnostics/RELEASE-v1.1/daily-{}-diagnostics/reanalysis/ERA5/F320/{}/v1.1.zarr'.format(variable, file_var_name))

    city_trend_error = {}
    city_absolute_error = {}
    for city in var_hist_clean.city.values:
        
        if variable == 'precip':
            var = 'pr'
        else:
            var = variable 

        sim_trend = (var_ssp370_clean.sel(city=city)[var].sel(time=slice('2080', '2099')).chunk(dict(time=-1)).groupby('time.season').quantile(q).values 
                     - var_hist_clean.sel(city=city)[var].sel(time=slice('1995', '2014')).chunk(dict(time=-1)).groupby('time.season').quantile(q).values)

        downscale_trend = (var_ssp370_ds.sel(city=city)[var].sel(time=slice('2080', '2099')).chunk(dict(time=-1)).groupby('time.season').quantile(q).values 
                           - var_hist_ds.sel(city=city)[var].sel(time=slice('1995', '2014')).chunk(dict(time=-1)).groupby('time.season').quantile(q).values)

        city_error = np.median(np.abs(downscale_trend - sim_trend))
        city_trend_error[city] = city_error

        sim_hist = (var_hist_ds.sel(city=city)[var].sel(time=slice('1995', '2014')).chunk(dict(time=-1)).groupby('time.season').quantile(q).values 
                    - var_ref.sel(city=city)[var].sel(time=slice('1995', '2014')).chunk(dict(time=-1)).groupby('time.season').quantile(q).values)

        abs_error = np.median(np.abs(sim_hist))

        city_absolute_error[city] = abs_error
        
    
        axes[i].plot(city_absolute_error[city], city_trend_error[city], 'o', c=next(clrs), label=city)
    axes[i].set_xlabel('Bias adjustment')
    axes[i].set_xlim([0, 2.5])
    axes[i].set_ylim([0, 2.5])
    axes[0].set_ylabel('Trend preservation')
    
    axes[4].legend(loc='center left', bbox_to_anchor=(1, 0.5))
    
    axes[i].set_title('{}'.format(q))

In [ ]:
var_ssp370_ds = xr.open_zarr('gs://downscaled-48ec31ab/diagnostics/RELEASE-v1.1/daily-{}-diagnostics/ScenarioMIP/BCC/BCC-CSM2-MR/ssp370/r1i1p1f1/day/{}/v1.1.zarr').format(variable, variable)
var_hist_ds = xr.open_zarr('gs://downscaled-48ec31ab/diagnostics/RELEASE-v1.1/daily-{}-diagnostics/CMIP/BCC/BCC-CSM2-MR/historical/r1i1p1f1/day/{}/v1.1.zarr').format(variable, variable)

var_ssp370_clean = xr.open_zarr('gs://downscaled-48ec31ab/diagnostics/RELEASE-v1.1/clean-daily-{}-diagnostics/ScenarioMIP/BCC/BCC-CSM2-MR/ssp370/r1i1p1f1/day/{}/v1.1.zarr').format(variable, variable)
var_hist_clean = xr.open_zarr('gs://downscaled-48ec31ab/diagnostics/RELEASE-v1.1/clean-daily-{}-diagnostics/CMIP/BCC/BCC-CSM2-MR/historical/r1i1p1f1/day/{}/v1.1.zarr').format(variable, variable)

var_ref = xr.open_zarr('gs://downscaled-48ec31ab/diagnostics/RELEASE-v1.1/daily-{}-diagnostics/reanalysis/ERA5/F320/{}/v1.1.zarr').format(variable, variable)

In [ ]:
fig, axes = plt.subplots(1, 1, figsize=(6, 4))

variable = 'tasmax'
    
if variable == "precip":
    file_var_name = "pr"
else:
    file_var_name = variable

var_ssp370_ds = xr.open_zarr('gs://downscaled-48ec31ab/diagnostics/RELEASE-v1.1/daily-{}-diagnostics/ScenarioMIP/BCC/BCC-CSM2-MR/ssp370/r1i1p1f1/day/{}/v1.1.zarr'.format(variable, file_var_name))
var_hist_ds = xr.open_zarr('gs://downscaled-48ec31ab/diagnostics/RELEASE-v1.1/daily-{}-diagnostics/CMIP/BCC/BCC-CSM2-MR/historical/r1i1p1f1/day/{}/v1.1.zarr'.format(variable, file_var_name))

var_ssp370_clean = xr.open_zarr('gs://downscaled-48ec31ab/diagnostics/RELEASE-v1.1/clean-daily-{}-diagnostics/ScenarioMIP/BCC/BCC-CSM2-MR/ssp370/r1i1p1f1/day/{}/v1.1.zarr'.format(variable, file_var_name))
var_hist_clean = xr.open_zarr('gs://downscaled-48ec31ab/diagnostics/RELEASE-v1.1/clean-daily-{}-diagnostics/CMIP/BCC/BCC-CSM2-MR/historical/r1i1p1f1/day/{}/v1.1.zarr'.format(variable, file_var_name))

var_ref = xr.open_zarr('gs://downscaled-48ec31ab/diagnostics/RELEASE-v1.1/daily-{}-diagnostics/reanalysis/ERA5/F320/{}/v1.1.zarr'.format(variable, file_var_name))

city_trend_error = {}
city_absolute_error = {}
for city in var_hist_clean.city.values:

    if variable == 'precip':
        var = 'pr'
    else:
        var = variable 
    q = 0.95
    sim_trend = (var_ssp370_clean.sel(city=city)[var].sel(time=slice('2080', '2099')).groupby('time.year').quantile(q).values 
                 - var_hist_clean.sel(city=city)[var].sel(time=slice('1995', '2014')).groupby('time.year').quantile(q).values)

    downscale_trend = (var_ssp370_ds.sel(city=city)[var].sel(time=slice('2080', '2099')).groupby('time.year').quantile(q).values 
                       - var_hist_ds.sel(city=city)[var].sel(time=slice('1995', '2014')).groupby('time.year').quantile(q).values)

    city_error = np.median(np.abs(downscale_trend - sim_trend))
    city_trend_error[city] = city_error

    sim_hist = (var_hist_ds.sel(city=city)[var].sel(time=slice('1995', '2014')).groupby('time.year').quantile(q).values 
                - var_ref.sel(city=city)[var].sel(time=slice('1995', '2014')).groupby('time.year').quantile(q).values)

    abs_error = np.median(np.abs(sim_hist))

    city_absolute_error[city] = abs_error


    axes.plot(city_absolute_error[city], city_trend_error[city], 'o', c=next(clrs), label=city)
axes.set_xlabel('Bias adjustment')
axes.set_ylabel('Trend preservation')

axes.legend(loc='center left', bbox_to_anchor=(1, 0.5))

axes.set_title(variable)


In [ ]:
city_trend_error